In [45]:
PRIME = 3*2**30+1
FIELD = GF(PRIME)
F.<x> = PolynomialRing(GF(PRIME),'x')
gen8192 = 1734477367
gen1024 = gen8192**8
gen_ = FIELD.gen
Y_INDEX = 6


In [46]:
def traceCalculator(a: int, trance_len: int):
    trace = [1, a]
    for i in range(2,trance_len):
        trace.append((trace[i-1] + trace[i-2])%PRIME)
    return trace

In [47]:
def polynomialEvaluation (trace, generator):
    points =[]
    for i, y in enumerate (trace): 
        points.append((generator**i, y))
    
    R = FIELD['x']
    polynomial = R.lagrange_polynomial(points)
    return polynomial

In [48]:
a=2
trance_len = 8
trace = traceCalculator(a=a,trance_len=trance_len)
#print(trace)
poly = polynomialEvaluation(trace=trace, generator=gen1024)
# Sainaty check:
assert poly(gen1024**Y_INDEX) == trace[Y_INDEX]
Y=trace[Y_INDEX]

In [49]:
def compositonPolynomial(poly, trance_len, Y, index_y, gen):
    n=trance_len
    p1 = (poly-1)/(x-gen**0)
    co1 = FiatShamirRandom(p1)
    p2 = (poly - Y)/(x-gen**index_y)
    co2 = FiatShamirRandom(data=p2)
    
    # (x-g**0)(x-g**1)...(x-g**(n-1)) = x**n-1
    # 
    constrain_3_numer = poly(gen ** 2 * x) - poly(gen * x) - poly(x)
    # constrain_3_numer should divide by all powers of gen: x=g**0, .... x= g**(n-3)
    constrain_3_denom = (x**n-1) / ( (x-gen**(n-1)) * (x-gen**(n-2)) )
    # p3 = (poly(gen ** 2 * x) - poly(gen * x) - poly(x))*(x-gen**(n-1)) \
    #     * (x-gen**(n-2))*(x-gen**(n-3))/(x**n-1)
    p3 = constrain_3_numer / constrain_3_denom
    co3 = FiatShamirRandom(p3)
    return p1*co1 + p2+co2 + p3*co3

In [201]:
def merkle(points: list):
    if len(points) <= 1:
        return points[0]
    squeezed_points = []
    for i in range(len(points)//2):
        temp_string = ''.join(str(points[i*2]))
        temp_string = temp_string.join(str(points[i*2+1]))
        squeezed_points.append(sha3(temp_string))
    if len(points)%2 == 1:
        squeezed_points.append(sha3(''.join(str(points[-1]))))
    return merkle(squeezed_points)

In [199]:
def sha3(string):
    byte=string.encode('ascii')
    m = hashlib.sha3_256()
    m.update(byte)
    
    return m.hexdigest()

In [200]:
def FiatShamirRandom(data: str)-> int:
    rand = int.from_bytes(data.encode('ascii'), "big") 
    return rand%PRIME

In [202]:
points = traceCalculator(2, 15)
merkle(points)

'bf356b7ff2a1dbdfed697f58d8549ae73f9894b74eaba65b21d6d543af0a2647'